## Step 0: 라이브러리 설치 및 Athena 연결

In [ ]:
!python -m ensurepip --upgrade
!pip install "sqlalchemy" --quiet
!pip install "boto3~=1.34"  --quiet
!pip install "jinja2" --quiet
!pip install "botocore" --quiet
!pip install "pandas" --quiet
!pip install "PyAthena" --quiet
!pip install "faiss-cpu" --quiet

In [ ]:
import json
import boto3
import sys

#sys.path.append('../')
from libs.din_sql import din_sql_lib as dsl

In [ ]:
ATHENA_RESULTS_S3_LOCATION = 'txt2sql-2-us-west-2-389394968670-workshop/athena_results/' # available in cloudformation outputs
ATHENA_CATALOG_NAME = 'txt2sql-2-tpc_ds' # available in cloudformation outputs
DB_NAME = "tpcds1"

In [ ]:
from libs.din_sql import din_sql_lib as dsl

#model_id = 'anthropic.claude-3-haiku-20240307-v1:0'
model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'

din_sql = dsl.DIN_SQL(bedrock_model_id=model_id)

In [ ]:
din_sql.athena_connect(catalog_name=ATHENA_CATALOG_NAME, 
               db_name=DB_NAME, 
               s3_prefix=ATHENA_RESULTS_S3_LOCATION)

## Step 1: 프롬프트 구성

In [ ]:
return_sql= din_sql.find_fields(db_name=DB_NAME)
print(return_sql)

In [ ]:
import os
import os
import jinja2 as j

question = "Which customer spent the most money in the web store?"

instructions_tag_start = '<instructions>'
instructions_tag_end = '</instructions>'
example_tag_start = '<example>'
example_tag_end = '</example>'
sql_tag_start = '```sql'
sql_tag_end = '```'

#current_dir = os.path.dirname(__file__)
JINJA_ENV = j.Environment(
    loader=j.FileSystemLoader("./libs/din_sql/prompt_templates"),
    autoescape=j.select_autoescape(
        enabled_extensions=('jinja'),
        default_for_string=True,
    )
)
easy_prompt = JINJA_ENV.get_template('basic_prompt.txt.jinja')
prompt = easy_prompt.render(
    instruction_tag_start=instructions_tag_start,
    instruction_tag_end=instructions_tag_end,
    fields=return_sql,
    example_tag_start=example_tag_start,
    example_tag_end=example_tag_end,
    test_sample_text=question,
    sql_tag_start=sql_tag_start,
    sql_tag_end=sql_tag_end
)
print(prompt)

## Step 2: LLM을 사용해 쿼리 생성

In [ ]:
import json
import boto3

bedrock_client = boto3.client(service_name='bedrock-runtime')

def llm_generation(prompt, stop_sequences=[], word_in_mouth=None):
    user_message =  {"role": "user", "content": prompt}
    messages = [user_message]
    if word_in_mouth:
        messages.append({
            "role": "assistant",
            "content": word_in_mouth,
        })
    response = bedrock_client.invoke_model(
        modelId=model_id,
        body=json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "messages": messages,
            "temperature": 0,
            "max_tokens": 8000,
            "stop_sequences": stop_sequences,
            })
    )
    response_dict = json.loads(response.get('body').read().decode("utf-8"))
    results = response_dict["content"][0]["text"]
    return results

sql_qry = llm_generation(prompt)

In [ ]:
SQL = sql_qry.split('```sql')[1].split('```')[0].strip()
print(f"{SQL}")

## Step 3: Athena 쿼리 실행

In [ ]:
import pandas as pd
result_set = din_sql.query(SQL)
pd.DataFrame(result_set)

## Step 4: 쿼리 결과를 활용해 답변 - 생략